# Welcome to Exkaldi

In this section, we will training a DNN acoustic model with Tensorflow 2.x.

If you want run this step, please install tensorflow firstly.

In [1]:
import exkaldi

import os
dataDir = os.path.join("..","examplesdata","librispeech_dummy")

We use keras to build and train model.

In [3]:
import tensorflow as tf
from tensorflow import keras
import random
import datetime
import numpy as np

Fix the random seed.

In [5]:
seed = 1
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

### Prepare Dataset

Restorage the training feature.

In [6]:
featFile = os.path.join(dataDir,"exp","mfcc.ark")

feat = exkaldi.load_feat(featFile)

feat = feat.to_numpy()

feat.dim

This feature is made following these steps:

compute mfcc (13) >> apply CMVN (13) >> add 2 order deltas (39) >> splice 1-1 frames (117)

We still further do global standerd normalization on it.

In [ ]:
feat = feat.normalize(std=True)

Them we load the alignment data. They have been generated in early step (07_train_triphone_HMM-GMM_delta).

We will use pdf-ID as target label. In exkaldi, transition-ID and phone-ID can also be extracted for mutiple tasks.

In [ ]:
aliFile = os.path.join(dataDir, "exp", "train_delta", "final.ali")
hmmFile = os.path.join(dataDir, "exp", "train_delta", "final.mdl")

ali = exkaldi.load_ali(aliFile)

ali = ali.to_numpy(aliType="pdfID", hmm=hmmFile)

ali

Look the classes of alignment. It is the output units of Neural Network.

In [ ]:
ali.classes

Then we tuple the feature and alignment in order to generate a dataset for Neural Network Framework. We use __tuple_data(...)__ function to group them. 

But note that, this function will group the achivements by their name, so please ensure their names are avaliable as python identifiers. (that means, we only allow lower and upper letters, digits, and underline in their names.)

In [ ]:
feat.rename("mfcc")
ali.rename("pdfID")

dataset = exkaldi.tuple_data([feat,ali], frameLevel=True)

datasetSize = len(dataset)
datasetSize

___dataset___ is a list. whose members are namedtuples. For example:

In [1]:
oneRecord = dataset[0]

oneRecord

NameError: name 'dataset' is not defined

Use name to get specified data.

In [ ]:
oneRecord.pdfID

If you train a sequential NN model, you may not want to tuple achivemnts data in framelevel but in utterance level. try to change the mode of tuple. 

You can tuple all kinds of exkaldi achivements such as feature, CMVN, alignment, probability. And even different feature such as MFCC, fBank and so on, different alignment such as PdfID, Phone ID. For example, now we want to do multiple tasks. 

In [ ]:
ali2 = exkaldi.load_ali(aliFile)

ali2 = ali2.to_numpy(aliType="phoneID", hmm=hmmFile)

ali2.rename("phoneID")

dataset2 = exkaldi.tuple_data([feat,ali,ali], frameLevel=True)

In [ ]:
dataset2[0]

In [ ]:
del ali2
del dataset2

del ali
del feat

### Training

Now we start to train DNN acoustic model. Fisrtly, design a data iterator from our provided dataset.

In [ ]:
def data_generater(dataset):
    length = len(dataset)
    while True:
        index = 0
        random.shuffle(dataset)
        while index < length:
            one = dataset[index]
            index += 1
            yield (one.mfcc[0], one.pdfID)

In [ ]:
batchSize = 64
tf_datasets = tf.data.Dataset.from_generator(
                                 lambda : data_generater(dataset),
                                 (tf.float32, tf.int32)
                            ).batch(batchSize).prefetch(batchSize)

Then define a simple Dense model.

In [ ]:
def make_DNN_model(inputsShape, classes):
    
    inputs = keras.Input(inputsShape)
    h1 = keras.layers.Dense(256, activation="relu", kernel_initializer="he_normal")(inputs)
    h1_bn = keras.layers.BatchNormalization()(h1)
    
    h2 = keras.layers.Dense(512, activation="relu", kernel_initializer="he_normal")(h1_bn)
    h2_bn = keras.layers.BatchNormalization()(h2)
    
    h3 = keras.layers.Dense(512, activation="relu", kernel_initializer="he_normal")(h2_bn)
    h3_bn = keras.layers.BatchNormalization()(h3)
    
    outputs = keras.layers.Dense(classes, use_bias=False)(h3_bn)
    
    return keras.Model(inputs, outputs)

In [ ]:
model = make_DNN_model((feat.dim,), ali.classes)

model.summary()

Here are optimizer and metrics.

In [ ]:
optimizer = keras.optimizers.Adam(0.001)

losses = keras.metrics.Mean(name="train/loss", dtype=tf.float32)
accs = keras.metrics.Mean(name="train/accuracy", dtype=tf.float32)

Speecify the output dir. You can use tensorboard to check the training results.

In [ ]:
outDir = os.path.join(dataDir, "exp", "train_DNN")

stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logDir = os.path.join(outDir, "log", stamp)
file_writer = tf.summary.create_file_writer(logDir)

In [ ]:
epochs = 20

epoch_iterations = datasetSize//batchSize
maxmum_iterations = epochs * epoch_iterations

epoch_iterations

In [ ]:
with file_writer.as_default():
    
    for batch in tf_datasets:
        data, label = batch
        with tf.GradientTape() as tape:
            
            logits = model(data, training=True)
            loss = keras.losses.sparse_categorical_crossentropy(label, logits, from_logits=True)
            losses(loss)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            
            pred = keras.backend.argmax(logits, axis=1)
            
            acc = exkaldi.nn.accuracy(label.numpy(), pred.numpy())
            accs(acc.accuracy)
        
        train_step = optimizer.iterations.numpy()
        if train_step % epoch_iterations == 0 or train_step > maxmum_iterations:
            current_loss = losses.result()
            current_acc = accs.result()
            tf.print( f"Step {train_step}/{maxmum_iterations}", f" Loss {current_loss:.6f}", f" Acc {current_acc:.6f}")
            
            tf.summary.scalar("train/loss", data=current_loss, step=train_step)
            tf.summary.scalar("train/accuracy", data=current_acc, step=train_step)
        
            if train_step > maxmum_iterations:
                tf.print( "Training Done" )
                break

Save the model in file.

In [10]:
tfModelFile = os.path.join(outDir, "dnn.h5")

model.save(tfModelFile, include_optimizer=False)

Now we predict the network output for test data for decoding. We do the same processing as training feature.

In [ ]:
testFeatFile = os.path.join(dataDir, "exp", "test_mfcc.ark")

testFeat = exkaldi.load_feat(testFeatFile)

testFeat = testFeat.to_numpy()

testFeat = testFeat.normalize(std=True)

In [14]:
prob = {}
for utt, mat in testFeat.items:
    logits = model(mat, training=False)
    prob[utt] = logits.numpy()

prob = exkaldi.load_prob(prob)

prob

___prob___ is an exkaldi __NumpyProbability__ object. Save it to file. We will decode it in the next step.

In [16]:
probFile = os.path.join(outDir, "amp.npy")

prob.save(probFile)

'../examplesdata/librispeech_light/exp/train_DNN/prob.npy'